In [1]:
from WindPy import w
from datetime import datetime, timedelta
w.start()
date = '2022-11-18'
code = ['005754.OF', '005756.OF', '008911.OF', '008913.OF', '007935.OF', '007936.OF', '008696.OF', '004827.OF', '006851.OF']
w.wsd(code[0], "sec_name,nav,NAV_adj_return1,return_1m,return_3m,return_1y", '2022-11-5', '2022-11-18', "annualized=1", usedf=True)[1]

Welcome to use Wind Quant API for Python (WindPy)!

COPYRIGHT (C) 2020 WIND INFORMATION CO., LTD. ALL RIGHTS RESERVED.
IN NO CIRCUMSTANCE SHALL WIND BE RESPONSIBLE FOR ANY DAMAGES OR LOSSES CAUSED BY USING WIND QUANT API FOR Python.


,SEC_NAME,NAV,NAV_ADJ_RETURN1,RETURN_1M,RETURN_3M,RETURN_1Y
2022-11-07,平安短债A,1.1488,0.008705,2.492929,1.922233,3.170184
2022-11-08,平安短债A,1.1486,-0.017409,2.283034,1.710601,3.124439
2022-11-09,平安短债A,1.1485,-0.008706,2.178233,1.710751,3.124719
2022-11-10,平安短债A,1.1482,-0.026121,1.031175,1.570145,3.070018
2022-11-11,平安短债A,1.1478,-0.034837,0.514333,1.500236,3.034111
2022-11-14,平安短债A,1.1463,-0.130685,-1.325670,0.974999,2.890225
2022-11-15,平安短债A,1.1453,-0.087237,-2.334451,0.416772,2.791240
2022-11-16,平安短债A,1.1443,-0.087313,-3.333787,-0.034663,2.683058
2022-11-17,平安短债A,1.1436,-0.061173,-4.420734,-0.277055,2.601830
2022-11-18,平安短债A,1.1438,0.017489,-4.321913,-0.242434,2.601363


In [1]:
from WindPy import w
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
try:
    from typing import Literal
except ImportError:
    from typing_extensions import Literal
from dateutil.relativedelta import relativedelta

In [5]:

class Wind_Exporter:
    """
    Wind_Exporter
    =====
    code: list
        股票代码, 例如['000001.SZ', '000002.SZ']
    indicator: str
        指标, 例如'close'
    method: str (default: 'wsd')
        wsd: 单日期 
        wss: 日期截面数据
    Date_List: list (default: None)
        日期列表, 例如['2020-01-01', '2020-01-02']
    StartDate: str (default: None)
        起始日期, 例如'2020-01-01'
    EndDate: str (default: None)
        结束日期, 例如'2020-01-02'
    options: str (default: None)
        选项, 遵循Wind API的options参数
    """

    def __init__(self, code=None, indicator: str = None, method: Literal['wsd', 'wss'] = 'wsd', 
                    Date_List: list = None, StartDate: Literal['before1m', 'before1y', None] = None, EndDate=None, options=None):
        self.method = method
        self.code = code if isinstance(code, list) else [
            i for i in code.split(',')]
        self.EndDate = EndDate if EndDate else datetime.today().strftime('%Y-%m-%d')
        self.Date_List = Date_List
        self.StartDate = EndDate if StartDate is None else StartDate
        self.set_date()
        self.indicator = indicator
        self.options = options
        self.data = []

    def __len__(self):
        return len(self.data)

    def __repr__(self):
        return f'Wind_Exporter({self.code}, {self.indicator}, {self.method}, {self.Date_List}, {self.StartDate}, {self.EndDate}, {self.options}), With data length {len(self)}'

    def check_connection(func):
        if not w.isconnected():
            w.start()
        return func

    @staticmethod
    def create_date_col(df, date):
        df['date'] = date
        column = np.roll(np.arange(len(df.columns)), 1)
        column[0], column[1] = column[1], column[0]
        return df.iloc[:, column]
    

#my_dict = {'code':['005754.OF', '005756.OF', '008911.OF', '008913.OF', '007935.OF', '007936.OF', '008696.OF', '004827.OF', '006851.OF'],
            # 'indicator':"sec_name,nav_date,nav,NAV_adj_return1,return_1m,return_3m,return_1y", 'options':"annualized=1", 'method':"wsd"}
    @classmethod
    def from_dict(cls, data: dict):
        return cls(**data) 
        

    def set_date(self):
        date_before_1m = (datetime.strptime(self.EndDate, '%Y-%m-%d') -
                          relativedelta(months=1) + relativedelta(days=1)).strftime('%Y%m%d')
        date_before_1y = (datetime.strptime(self.EndDate, '%Y-%m-%d') -
                          relativedelta(years=1) + relativedelta(days=1)).strftime('%Y%m%d')
        if self.StartDate == 'before1m':
            self.StartDate = date_before_1m
        elif self.StartDate == 'before1y':
            self.StartDate = date_before_1y
        else:
            pass

    @check_connection
    def get_data(self, output: Literal['df', 'excel'] = None, round_=None):
        if self.method == 'wsd':
            self.get_data_wsd()
        elif self.method == 'wss':
            self.get_data_wss()
        else:
            raise ValueError('method must be wsd or wss')

        if round_ is not None:
            for i in range(len(self.data)):
                self.data[i] = self.data[i].round(round_)
        if output == 'df':
            return self.data
        elif output == 'excel':
            pass
        else:
            return self

    def get_data_wsd(self):
        if self.Date_List is None:
            if self.StartDate == self.EndDate:
                multi_data = []
                for i in self.code:
                    data_ = w.wsd(i, self.indicator, self.StartDate,
                                  self.EndDate, options=self.options, usedf=True)
                    multi_data.append(data_[1])
                self.data.append(pd.concat(multi_data, axis=0))
            else:
                multi_data = []
                for i in self.code:
                    data_ = w.wsd(i, self.indicator, self.StartDate,
                                  self.EndDate, options=self.options, usedf=True)
                    multi_data.append(data_[1])
                self.data.append(multi_data)
        else:
            for date in self.Date_List:
                multi_data = []
                for i in self.code:
                    data_ = w.wsd(i, self.indicator, date, date,
                                  options=self.options, usedf=True)
                    multi_data.append(data_[1])
                self.data.append(pd.concat(multi_data, axis=0))
        return self

# w.wss("002450.OF","NAV_adj_return","startDate=20221117;endDate=20221118")
# w.wss("009878.OF", "sec_name","startDate=20221117;endDate=20221118")

    @check_connection
    def combine_wss(self, group_data: list):
        for i in group_data:
            data_ = []
            for j in i:
                data_.append(j[1])
            data_ = pd.concat(data_, axis=1)
            self.data.append(data_)
        return self

    def get_data_wss(self):
        self.StartDate = self.StartDate.replace('-', '')
        self.EndDate = self.EndDate.replace('-', '')
        args = [self.options, self.StartDate, self.EndDate]
        if self.options is None:
            args = args[1:]
        options = f'startDate={args[0]};endDate={args[1]};tradeDate={args[1]}' if len(
            args) == 2 else f'{args[0]};startDate={args[1]};endDate={args[2]};tradeDate={args[2]}'
        print(options)
        data_ = w.wss(self.code, self.indicator, options=options, usedf=True)
        self.data.append(data_[1])
        return self

    def add_data(self, we_obj, method: Literal['concat', 'append'] = 'concat', round_=None):
        if method == 'concat':
            we_obj.get_data(round_=round_)
            self.data[-1] = pd.concat([self.data[-1], we_obj.data[-1]], axis=1)
        elif method == 'append':
            we_obj.get_data(round_=round_)
            for i in we_obj.data:
                self.data.append(i)
        else:
            raise ValueError('method must be concat or append')
        return self

    def excel_export(self, path=None, sheet_name: list = None, column_name: list = None):
        """
        导出为Excel
        =====
        path: str (default: None)
            导出路径, 例如'./data.xlsx', 默认为None, 会在output目录下生成{日期/开始日期_结束日期}.xlsx
        sheet_name: list (default: None)
            sheet名称, 例如['sheet1', 'sheet2'], 默认为None, 会使用工作簿数量作为sheet名称
        column_name: list (default: None)
            列名称, 二维数组, 例如[['col1', 'col2'], ['col1', 'col2']], 默认为None, 会使用Wind默认列名称
        """
        if path is None:
            if self.Date_List is None:
                if self.StartDate == self.EndDate:
                    path = f'./output/{self.EndDate}.xlsx'
                else:
                    path = f'./output/{self.StartDate}_{self.EndDate}.xlsx'
            else:
                path = f'./output/{self.Date_List[0]}.xlsx'
        if sheet_name is None:
            sheet_name = np.arange(len(self.data)) + 1
        if column_name is not None:
            try:
                _ = column_name[0][0]
            except:
                raise ValueError('column_name 应当是二维列表')
            for i, data in enumerate(self.data):
                try:
                    data.columns = column_name[i]
                    print(column_name[i])
                    print('更名完成')
                except:
                    raise ValueError('列名数与数据不匹配')
        with pd.ExcelWriter(path, datetime_format='YYYY-MM-DD') as writer:
            for i in enumerate(self.data):
                i[1].to_excel(writer, sheet_name=sheet_name[i[0]])
        return self

Welcome to use Wind Quant API for Python (WindPy)!

COPYRIGHT (C) 2020 WIND INFORMATION CO., LTD. ALL RIGHTS RESERVED.
IN NO CIRCUMSTANCE SHALL WIND BE RESPONSIBLE FOR ANY DAMAGES OR LOSSES CAUSED BY USING WIND QUANT API FOR Python.


In [6]:
code_for_zhai = ['005754.OF', '005756.OF', '008911.OF', '008913.OF', '007935.OF', '007936.OF', '008696.OF', '004827.OF', '006851.OF']
code_for_ETF = ['516760.OF', '516820.OF', '515700.OF', '561600.OF']
code_cundan_str = """015645.OF,015644.OF,015826.OF,015862.OF,014437.OF,015823.OF,015648.OF,014427.OF,015822.OF,015647.OF,015875.OF,
                        014426.OF,015864.OF,015861.OF,015646.OF,015643.OF,014430.OF,015956.OF,015825.OF,014428.OF,015827.OF,015863.OF,014429.OF,015944.OF,015955.OF,016082.OF,016063.OF,016083.OF"""
code_for_meiri_str = """002450.OF,004827.OF,015645.OF,008694.OF,005754.OF,700003.OF,000739.OF,007935.OF,009661.OF,009878.OF,
                        010126.OF,014460.OF,013767.OF,013687.OF,004390.OF,012475.OF,007893.OF,885001.WI"""

choice_data = "2022-11-24"
a = Wind_Exporter(code=code_for_zhai, indicator="sec_name,nav_date,nav,NAV_adj_return1,return_1m,return_3m,return_1y",options="annualized=1",method="wsd",EndDate=choice_data)
b = Wind_Exporter(code=code_for_ETF, indicator="sec_name,nav_date,nav,NAV_adj_return1,return_1w,return_1m",options="annualized=1",method="wsd",EndDate=choice_data)
c = Wind_Exporter(code="015645.OF", indicator="sec_name,nav_date,nav,NAV_adj_return1,return_1w,return_1m",options="annualized=1",method="wsd",EndDate=choice_data)
d = Wind_Exporter(code=code_cundan_str, indicator="sec_name,nav_date,return,risk_annualintervalyield,issue_date,fund_setupdate",options="annualized=0",method="wss",EndDate=choice_data)
e = Wind_Exporter(code=code_for_meiri_str, indicator="sec_name,nav_date,nav,NAV_adj_return1,NAV_adj_return,return_ytd",options="annualized=0",method="wss",StartDate="before1m",EndDate=choice_data)
e_ = Wind_Exporter(code=code_for_meiri_str, indicator="NAV_adj_return",method="wss",StartDate="before1y",EndDate=choice_data)
a.get_data(round_=4).add_data(b, method='append',round_=4).add_data(c, method='append',round_=4).add_data(d, method='append',round_=4).add_data(e, method='append',round_=4).add_data(e_, method='concat',round_=4)


annualized=0;startDate=20221124;endDate=20221124;tradeDate=20221124
annualized=0;startDate=20221025;endDate=20221124;tradeDate=20221124
startDate=20211125;endDate=20221124;tradeDate=20221124


Wind_Exporter(['005754.OF', '005756.OF', '008911.OF', '008913.OF', '007935.OF', '007936.OF', '008696.OF', '004827.OF', '006851.OF'], sec_name,nav_date,nav,NAV_adj_return1,return_1m,return_3m,return_1y, wsd, None, 2022-11-24, 2022-11-24, annualized=1), With data length 5

In [ ]:
a.data[3]

In [52]:
a.excel_export(sheet_name=['债', 'ETF', '同业', '存单', '每日'], column_name=[['证券简称', '基金净值日期','单位净值', '当期复权单位净值增长率', '近1月回报', '近3月回报', '近1年回报'],
                ['证券简称', '基金净值日期','单位净值', '当期复权单位净值增长率', '近1周回报', '近1月回报'],['证券简称', '基金净值日期','单位净值', '当期复权单位净值增长率', '近1周回报', '近1月回报'],
                ['证券简称', '基金净值日期','区间回报', '区间收益率', '发行日期', '基金成立日'], ['证券简称', '基金净值日期','单位净值', '当期复权单位净值增长率', '复权单位净值增长率(截止日1月前)', '今年以来回报', '复权单位净值增长率(截止日1年前)']],
                path='./output/{}.xlsx'.format(choice_data))

Wind_Exporter(['005754.OF', '005756.OF', '008911.OF', '008913.OF', '007935.OF', '007936.OF', '008696.OF', '004827.OF', '006851.OF'], sec_name,nav_date,nav,NAV_adj_return1,return_1m,return_3m,return_1y, wsd, None, 2022-11-24, 2022-11-24, annualized=1), With data length 5

In [1]:
from wdpcore import Wind_Exporter
# code = [[w.wss("516760.OF,516820.OF,515700.OF,561600.OF", "swing","tradeDate=20221122;cycle=D", usedf=True)],
# [w.wss("002450.OF","NAV_adj_return","startDate=20221117;endDate=20221118", usedf=True),w.wss("009878.OF", "sec_name","startDate=20221117;endDate=20221118", usedf=True)]]
code = ['002450.OF','009878.OF']
code_for_meiri_str = """002450.OF,004827.OF,015645.OF,008694.OF,005754.OF,700003.OF,000739.OF,007935.OF,009661.OF,009878.OF,010126.OF,014460.OF,013767.OF,013687.OF,004390.OF,012475.OF,007893.OF,885001.WI"""
# a = Wind_Exporter(method='wss', code=code_for_meiri_str, indicator="sec_name,nav_date,nav,NAV_adj_return1,NAV_adj_return,return_ytd",StartDate='before1m',EndDate='2022-11-24',options="annualized=0")
b = Wind_Exporter(method='wss', code=code_for_meiri_str, indicator="NAV_adj_return",StartDate="before1y",EndDate='2022-11-24')
b.get_data()


Welcome to use Wind Quant API for Python (WindPy)!

COPYRIGHT (C) 2020 WIND INFORMATION CO., LTD. ALL RIGHTS RESERVED.
IN NO CIRCUMSTANCE SHALL WIND BE RESPONSIBLE FOR ANY DAMAGES OR LOSSES CAUSED BY USING WIND QUANT API FOR Python.


Wind_Exporter(['002450.OF', '004827.OF', '015645.OF', '008694.OF', '005754.OF', '700003.OF', '000739.OF', '007935.OF', '009661.OF', '009878.OF', '010126.OF', '014460.OF', '013767.OF', '013687.OF', '004390.OF', '012475.OF', '007893.OF', '885001.WI'], NAV_adj_return, wss, None, 20211125, 20221124, None), With data length 1

In [2]:
b.excel_export()

Wind_Exporter(['002450.OF', '004827.OF', '015645.OF', '008694.OF', '005754.OF', '700003.OF', '000739.OF', '007935.OF', '009661.OF', '009878.OF', '010126.OF', '014460.OF', '013767.OF', '013687.OF', '004390.OF', '012475.OF', '007893.OF', '885001.WI'], NAV_adj_return, wss, None, 20211125, 20221124, None), With data length 1

In [11]:
import datetime as dt
dt.datetime.today().strftime('%Y-%m-%d')

'2022-12-01'

In [5]:
import datetime
from WindPy import w
today = datetime.date.today() 
w.tdaysoffset(-5, today, "Period=D;Days=Trading")

.ErrorCode=0
.Codes=[]
.Fields=[]
.Times=[20221124]
.Data=[[2022-11-24 00:00:00]]

In [15]:
import datetime
from WindPy import w
today = datetime.date.today() 
w.tdaysoffset(-10, today, "Period=D;Days=Trading").Data

[[datetime.datetime(2022, 11, 17, 0, 0)]]

In [5]:
b.EndDate

'20221124'

In [33]:
a.add_data(b, method='concat',round_=4)


startDate=20211125;endDate=20221124;tradeDate=20221124


Wind_Exporter(['002450.OF', '004827.OF', '015645.OF', '008694.OF', '005754.OF', '700003.OF', '000739.OF', '007935.OF', '009661.OF', '009878.OF', '010126.OF', '014460.OF', '013767.OF', '013687.OF', '004390.OF', '012475.OF', '007893.OF', '885001.WI'], sec_name,nav_date,nav,NAV_adj_return1,NAV_adj_return,return_ytd, wss, None, 20221025, 20221124, annualized=0), With data length 1

In [1]:
from WdpCore import Wind_Exporter
code_for_zhai = ['005754.OF', '005756.OF', '008911.OF', '008913.OF', '007935.OF', '007936.OF', '008696.OF', '004827.OF', '006851.OF']
a = Wind_Exporter(code=code_for_zhai, indicator="sec_name,nav_date,nav,NAV_adj_return1,return_1m,return_3m,return_1y",options="annualized=1",method="wsd",StartDate="-21TD")
a.get_data(round_=4)

Welcome to use Wind Quant API for Python (WindPy)!

COPYRIGHT (C) 2020 WIND INFORMATION CO., LTD. ALL RIGHTS RESERVED.
IN NO CIRCUMSTANCE SHALL WIND BE RESPONSIBLE FOR ANY DAMAGES OR LOSSES CAUSED BY USING WIND QUANT API FOR Python.


Wind_Exporter(['005754.OF', '005756.OF', '008911.OF', '008913.OF', '007935.OF', '007936.OF', '008696.OF', '004827.OF', '006851.OF'], sec_name,nav_date,nav,NAV_adj_return1,return_1m,return_3m,return_1y, wsd, None, -21TD, 2022-12-02, annualized=1), With data length 9

In [2]:
from WdpCore import Wind_Exporter
code_for_zhai = ['005754.OF', '005756.OF', '007935.OF', '007936.OF']
info = [
"""购买信息：
 A类(005754)申购费率0.10%-0.30%，赎回费率0.00%-1.50%（>30天赎回费率=0）""",
"""购买信息：
 E类(005756)销售服务费率0.25%（每年），赎回费率0.00%-1.50%（＞30天赎回费率=0）""",
"""购买信息：
 A类(007935)申购费率：M＜100万，费率0.8%；100万≤M＜300万，费率0.50%；300万≤M＜500万，费率0.30%；M≥500万，每笔1000元；赎回费率：0.00%-1.50%（≥30天赎回费率=0）""",
"""购买信息：
 C类(007936)销售服务费率0.50%（每年）；赎回费率：0.00%-1.50%（≥30天赎回费率=0）"""]
a = Wind_Exporter(code=code_for_zhai, indicator="sec_name,nav_date,nav,NAV_adj_return1,return_1y",options="annualized=1",method="wsd",StartDate="-21TD")
a.get_data()

Wind_Exporter(['005754.OF', '005756.OF', '007935.OF', '007936.OF'], sec_name,nav_date,nav,NAV_adj_return1,return_1y, wsd, None, -21TD, 2022-12-02, annualized=1), With data length 4

In [3]:
data = []
for i,code in enumerate(code_for_zhai):
    b = a.data[i].sort_values(by='NAV_DATE', ascending=False)
    name = b['SEC_NAME'].iloc[0]
    b.NAV_ADJ_RETURN1 = (b.NAV_ADJ_RETURN1/100).round(4)
    b.RETURN_1Y = (b.RETURN_1Y/100).round(4)
    data.append([b,name,code])

In [16]:
import xlwings as xw
import datetime as dt
# invisible_app = xw.App(visible=False)
# book = invisible_app.books.open('./template/zhai_tem.xlsx')
book = xw.Book('./template/zhai2_tem.xlsx')

In [17]:
for i in range(len(data)//2):
    i_ = i*2
    sheet_ = book.sheets[0]
    sheet_.name = data[i_][1]
    sheet_["A4"].options(index=False, header=False).value = data[i_][0].sort_values(by='NAV_DATE', ascending=False).loc[:,["NAV_DATE","NAV","NAV_ADJ_RETURN1"]]
    sheet_["F4"].options(index=False, header=False).value = data[i_][0].sort_values(by='NAV_DATE', ascending=False).loc[:,["RETURN_1Y"]]
    sheet_["A1"].value = f"{data[i_][1]}（{data[i_][2].replace('.OF','')}）收益情况播报"
    sheet_["A26"].value = info[i]

    sheet_["A31"].options(index=False, header=False).value = data[i_+1][0].sort_values(by='NAV_DATE', ascending=False).loc[:,["NAV_DATE","NAV","NAV_ADJ_RETURN1"]]
    sheet_["F31"].options(index=False, header=False).value = data[i_+1][0].sort_values(by='NAV_DATE', ascending=False).loc[:,["RETURN_1Y"]]
    sheet_["A28"].value = f"{data[i_+1][1]}（{data[i_+1][2].replace('.OF','')}）收益情况播报"
    sheet_["A53"].value = info[i+1]
    sheet_.copy()
# book.save(f"./output/债券基金收益情况播报_{dt.datetime.now().strftime('%Y-%m-%d')}.xlsx")
# invisible_app.quit()

In [14]:
sheet_["A53"].rows.autofit()

In [ ]:
book.close()

In [58]:
for n,i in enumerate(data):
    sheet_ = book.sheets[0]

    sheet_.name = i[1]
    sheet_["A4"].options(index=False, header=False).value = i[0].sort_values(by='NAV_DATE', ascending=False).loc[:,["NAV_DATE","NAV","NAV_ADJ_RETURN1"]]
    sheet_["F4"].options(index=False, header=False).value = i[0].sort_values(by='NAV_DATE', ascending=False).loc[:,["RETURN_1Y"]]
    sheet_["A1"].value = f"{i[1]}（{i[2].replace('.OF','')}）收益情况播报"
    sheet_["A26"].value = info[n]
    sheet_.copy()
book.save(f"./output/债券基金收益情况播报_{dt.datetime.now().strftime('%Y-%m-%d')}.xlsx")
invisible_app.quit()

In [ ]:
str.replace(".OF","")

In [21]:
sheet1["A26"].value = """购买信息：
 A类(005754)申购费率0.10%-0.30%，赎回费率0.00%-1.50%（>30天赎回费率=0）"""

book.sheets


book.save('./1.xlsx')

In [26]:
book.sheets.


Sheets([<Sheet [1.xlsx]平安短债（近1年）>, <Sheet [1.xlsx]平安短债（近1年） (2)>, <Sheet [1.xlsx]平安短债（近1年） (3)>])

In [35]:
b

,SEC_NAME,NAV_DATE,NAV,NAV_ADJ_RETURN1,RETURN_1Y
2022-12-01,平安短债A,2022-12-01,1.1447,-0.0002,0.0255
2022-11-30,平安短债A,2022-11-30,1.1449,-0.0002,0.0258
2022-11-29,平安短债A,2022-11-29,1.1451,-0.0004,0.0260
2022-11-28,平安短债A,2022-11-28,1.1456,0.0002,0.0266
2022-11-25,平安短债A,2022-11-25,1.1454,0.0002,0.0267
2022-11-24,平安短债A,2022-11-24,1.1452,0.0008,0.0266
2022-11-23,平安短债A,2022-11-23,1.1443,0.0003,0.0259
2022-11-22,平安短债A,2022-11-22,1.1440,-0.0001,0.0258
2022-11-21,平安短债A,2022-11-21,1.1441,0.0003,0.0262
2022-11-18,平安短债A,2022-11-18,1.1438,0.0002,0.0260


In [55]:
import pandas as pd
import datetime as dt
import numpy as np
import xlwings as xw
date = dt.datetime.now() - dt.timedelta(days=1)
zhai_data = pd.read_excel(f'./output/{date.strftime("%Y-%m-%d")}.xlsx',sheet_name='每日',index_col=0)
zhai_data.iloc[:,[1]] = zhai_data.iloc[:,[1]].astype(np.datetime64)
etf_data = pd.read_excel(f'./output/{date.strftime("%Y-%m-%d")}.xlsx',sheet_name='ETF',index_col=0)
cun_data = pd.read_excel(f'./output/{date.strftime("%Y-%m-%d")}.xlsx',sheet_name='同业',index_col=0)

app1 = xw.App(visible=False)
app1.display_alerts = False
book = app1.books.open('./template/每日播报.xlsm')
book.sheets[0]["A2"].options(header=False).value = zhai_data
book.sheets[1]["A2"].options(header=False).value = etf_data
book.sheets[1]["K2"].value = cun_data["近1周回报"][0]
book.sheets[3].activate()
book.macro('Automatic.存单').run()
book.macro('Automatic.ETF').run()
book.macro('Automatic.Copy').run()

In [55]:
indicator = "sec_name,nav,NAV_adj_return1,return_1m,return_3m,return_1y"
date = ['2022-11-18', '2022-11-17']
code = ['005754.OF', '005756.OF', '008911.OF', '008913.OF', '007935.OF', '007936.OF', '008696.OF', '004827.OF', '006851.OF']
a = Wind_Exporter(code, method='wsd', indicator=indicator, StartDate='before1y', options="annualized=1").get_data()



In [95]:
import xlwings as xw
cun_data = pd.read_excel(f'./output/{date.strftime("%Y-%m-%d")}.xlsx',sheet_name='存单')
cun_data_ = cun_data.sort_values(by='区间收益率',ascending=False).iloc[:,[0,1,3,4]]
cun_data_['Rank'] = np.arange(1,len(cun_data_)+1)
cun_data_ = cun_data_.iloc[:,[4,0,1,2,3]]

In [101]:
app1 = xw.App(add_book=False)
app1.display_alerts = False
book = app1.books.open('./template/全市场同存排名.xlsx')
book.sheets[0]["A3"].options(index=False,header=False).value = cun_data_
i= int(book.sheets[0]["B34"].value)
book.sheets[0][f"A{i}:E{i}"].api.Interior.Color = 65535

In [104]:
xw.apps

Apps([<Excel App 33888>])

<Range [全市场同存排名.xlsx]全市场同存!$A$5:$E$5>

In [4]:
from WdpCore import Wind_Exporter
import datetime as dt
import xlwings as xw
date = dt.datetime.now() - dt.timedelta(days=1)
a = Wind_Exporter(code="015645.OF", indicator="nav_date,nav,NAV_adj_return1,return_1w,return_1m",options="annualized=1",method="wsd",StartDate="-22TD",EndDate=date.strftime("%Y-%m-%d"))
a.get_data()

Wind_Exporter(['015645.OF'], nav_date,nav,NAV_adj_return1,return_1w,return_1m, wsd, None, -22TD, 2022-12-01, annualized=1), With data length 1

In [5]:
b = a.data[0]
b.NAV_ADJ_RETURN1 = (b.NAV_ADJ_RETURN1/100).round(4)
b.RETURN_1W = (b.RETURN_1W/100).round(4)
day30 = b.RETURN_1M[-1]/100

In [6]:
b.iloc[:,[0,1,2]]
b.RETURN_1W
app1 = xw.App(add_book=False)
app1.display_alerts = False
book = app1.books.open('./template/平安同存收益率.xlsx')
sheet_ = book.sheets[0]
sheet_["A4"].options(index=False,header=False).value = b.iloc[:,[0,1,2]]
sheet_["F4"].options(index=False,header=False).value = b.RETURN_1W
sheet_["F2"].value = day30

In [31]:
import time
import os

def get_picture(app,sheet_,range_:str,picture_name:str):
    app.books.add()
    chart_ = app.books[-1].sheets[0].charts.add(left=0,top=0,width=sheet_[range_].width,height=sheet_[range_].height)
    time.sleep(1)
    sheet_[range_].api.CopyPicture(Appearance=1,Format=-4147)
    chart_.api[1].Paste()
    chart_.api[1].Export(f"{os.path.dirname(app.books[0].api.Path)}\\output\\{picture_name}.png")
    app.books[-1].close()
get_picture(app1,sheet_,"A1:F26","寄了")

In [28]:
import os
os.path.dirname(book.api.Path)

'C:\\Users\\Light_ouo\\Documents\\Code\\Wind_Exporter_lite'

In [21]:
book.api.Path

'C:\\Users\\Light_ouo\\Documents\\Code\\Wind_Exporter_lite\\template'

In [14]:
chart_.api[1].Export("平安同存收益率.png")


True

In [10]:
from meiri_report import quit_app
import datetime as dt
import xlwings as xw
import pandas as pd
import numpy as np
import time
from pathlib import Path

def tongcun_rank_report(date=dt.datetime.now(), path='./output/'):
    cun_data = pd.read_excel(f'{path}/{date.strftime("%Y-%m-%d")}.xlsx',sheet_name='存单')
    cun_data_ = cun_data.sort_values(by='区间收益率',ascending=False).iloc[:,[0,1,3,4]]
    cun_data_['Rank'] = np.arange(1,len(cun_data_)+1)
    cun_data_ = cun_data_.iloc[:,[4,0,1,2,3]]
    app1 = xw.App(add_book=False)
    app1.display_alerts = False
    book = app1.books.open('./template/全市场同存排名.xlsx')
    book.sheets[0]["A3"].options(index=False,header=False).value = cun_data_
    i= int(book.sheets[0]["B34"].value)
    book.sheets[0][f"A{i}:E{i}"].api.Interior.Color = 65535
    book.sheets[0]["A1"].value = f"全市场同存排名({date.strftime('%Y')}年{date.strftime('%m')}月{date.strftime('%d')}日)"

    book.save(f'{path}/全市场同存排名{date.strftime("%Y-%m-%d")}.xlsx')
    time.sleep(3)
    get_picture(app1,book.sheets[0],"A1:E33",f"{date.strftime('%Y-%m-%d')}全市场同存排名",path)
    quit_app()  

def get_picture(app,sheet_,range_:str,picture_name:str,path:str):
    app.books.add()
    chart_ = app.books[-1].sheets[0].charts.add(left=0,top=0,width=sheet_[range_].width,height=sheet_[range_].height)
    print(range_)
    print(sheet_)
    print(chart_)
    sheet_[range_].api.CopyPicture(Appearance=1,Format=-4147)
    time.sleep(0.1)
    chart_.api[1].Paste()
    path = Path(path).resolve().parent
    print(f"{os.path.join(path, 'pic')}\\{picture_name}.png")
    chart_.api[1].Export(f"{os.path.join(path, 'pic')}\\{picture_name}.png")
    app.books[-1].close()
    print("-----图片生成done-----")


choice_data = dt.datetime.now()
path = './output/{}'.format(choice_data.strftime('%Y-%m-%d'))
path = path + '/data'

tongcun_rank_report(choice_data, path)

A1:E33
<Sheet [全市场同存排名2022-12-08.xlsx]全市场同存>
C:\Users\Light_ouo\Documents\Code\Wind_Exporter_lite\output\2022-12-08\pic\2022-12-08全市场同存排名.png


com_error: (-2147352567, '发生意外。', (0, None, None, None, 0, -2147287037), None)

In [25]:
choice_data = dt.datetime.now()
path = './output/{}'.format(choice_data.strftime('%Y-%m-%d'))
path = Path(path).resolve()
xw.books[0].sheets[0]["A1:E33"].api.CopyPicture(Appearance=1,Format=-4147)
chart_ = xw.books[1].sheets[0].charts[0]
print(f"{os.path.join(path, 'picture')}\\{1}.png")
chart_.api[1].Export(f"{os.path.join(path, 'picture')}\\{1}.png")

C:\Users\Light_ouo\Documents\Code\Wind_Exporter_lite\output\2022-12-08\picture\1.png


True

Charts([<Chart 'Chart 1' in <Sheet [工作簿1]Sheet1>>])

In [13]:
from WdpCore import Wind_Exporter
import xlwings as xw
import datetime as dt
import time
from meiri_report import quit_app,get_picture
def bond_report(date = dt.datetime.today()):
    code_for_zhai = ['005754.OF', '005756.OF', '007935.OF', '007936.OF']
    info = [
    """购买信息：
    A类(005754)申购费率0.10%-0.30%，赎回费率0.00%-1.50%（>30天赎回费率=0）""",
    """购买信息：
    E类(005756)销售服务费率0.25%（每年），赎回费率0.00%-1.50%（＞30天赎回费率=0）""",
    """购买信息：
    A类(007935)申购费率：M＜100万，费率0.8%；100万≤M＜300万，费率0.50%；300万≤M＜500万，费率0.30%；M≥500万，每笔1000元；赎回费率：0.00%-1.50%（≥30天赎回费率=0）""",
    """购买信息：
    C类(007936)销售服务费率0.50%（每年）；赎回费率：0.00%-1.50%（≥30天赎回费率=0）"""]


    # 1. 获取数据
    a = Wind_Exporter(code=code_for_zhai, indicator="sec_name,nav_date,nav,NAV_adj_return1,return_1y",options="annualized=1",method="wsd",StartDate="ED-21TD",EndDate=date.strftime("%Y-%m-%d"))
    a.get_data()
    data = []
    for i,code in enumerate(code_for_zhai):
        b = a.data[i].sort_values(by='NAV_DATE', ascending=False)
        name = b['SEC_NAME'].iloc[0]
        b.NAV_ADJ_RETURN1 = (b.NAV_ADJ_RETURN1/100).round(4)
        b.RETURN_1Y = (b.RETURN_1Y/100).round(4)
        data.append([b,name,code])

    print("数据获取完毕")

    # 2. 写入Excel
    invisible_app = xw.App(add_book=False)
    book = invisible_app.books.open('./template/zhai2_tem.xlsx')
    for i in range(len(data)//2):
        i_ = i*2
        sheet_ = book.sheets[0]
        sheet_.name = data[i_][1]
        sheet_["A4"].options(index=False, header=False).value = data[i_][0].sort_values(by='NAV_DATE', ascending=False).loc[:,["NAV_DATE","NAV","NAV_ADJ_RETURN1"]]
        sheet_["F4"].options(index=False, header=False).value = data[i_][0].sort_values(by='NAV_DATE', ascending=False).loc[:,["RETURN_1Y"]]
        sheet_["A1"].value = f"{data[i_][1]}（{data[i_][2].replace('.OF','')}）收益情况播报"
        sheet_["A26"].value = info[i_]

        sheet_["A31"].options(index=False, header=False).value = data[i_+1][0].sort_values(by='NAV_DATE', ascending=False).loc[:,["NAV_DATE","NAV","NAV_ADJ_RETURN1"]]
        sheet_["F31"].options(index=False, header=False).value = data[i_+1][0].sort_values(by='NAV_DATE', ascending=False).loc[:,["RETURN_1Y"]]
        sheet_["A28"].value = f"{data[i_+1][1]}（{data[i_+1][2].replace('.OF','')}）收益情况播报"
        sheet_["A53"].value = info[i_+1]
        sheet_.copy()
        time.sleep(3)
        get_picture(invisible_app,sheet_,"A1:F55",f"{date.strftime('%Y-%m-%d')}{data[i_][1]}")
    sheet_.delete()
    book.save(f"./output/债券基金收益情况播报_{date.strftime('%Y-%m-%d')}.xlsx")
    quit_app()
    print("数据写入完毕")



In [14]:
bond_report(dt.datetime.now()-dt.timedelta(days=1))

数据获取完毕
-----图片生成done-----
-----图片生成done-----
数据写入完毕


In [5]:
data_4 = pd.read_excel(f'./output/2022-12-05.xlsx',sheet_name='债',index_col=0).round(2)
[data_4['近1年回报'][f'{i}'] for i in ['005754.OF', '005756.OF']]

[2.54, 2.28]

In [6]:
data_4

,证券简称,基金净值日期,单位净值,当期复权单位净值增长率,近1月回报,近3月回报,近1年回报
005754.OF,平安短债A,2022-12-05,1.14,0.03,-3.85,-0.14,2.54
005756.OF,平安短债E,2022-12-05,1.13,0.03,-4.10,-0.42,2.28
008911.OF,平安元丰中短债A,2022-12-05,1.07,-0.04,-13.10,-4.17,1.37
008913.OF,平安元丰中短债E,2022-12-05,1.06,-0.05,-13.38,-4.45,1.11
007935.OF,平安惠澜纯债A,2022-12-05,1.13,-0.05,-9.62,-3.18,2.71
007936.OF,平安惠澜纯债C,2022-12-05,1.11,-0.06,-10.15,-3.64,2.20
008696.OF,平安元盛超短债E,2022-12-05,1.06,0.01,0.46,-0.75,1.41
004827.OF,平安中短债A,2022-12-05,1.14,0.00,-11.19,-3.60,2.07
006851.OF,平安中短债E,2022-12-05,1.13,-0.01,-11.47,-3.87,1.80


In [38]:
df = pd.DataFrame([(.21, 2.285), (.01, .67), (.66, .03), (.21, .18)],
                  columns=['dogs', 'cats'])
df.round(2)

,dogs,cats
0,0.21,2.28
1,0.01,0.67
2,0.66,0.03
3,0.21,0.18


In [18]:
round(2.15, 1)


2.1

In [19]:
round(5.15, 1)

5.2

In [34]:
round(2.285, 2)

2.29

In [2]:
from decimal import Decimal,ROUND_HALF_UP
def round_half_up(number, ndigits):
    if isinstance(number, list):
        return [Decimal(str(num)).quantize(Decimal('0.' + '0' * ndigits), rounding=ROUND_HALF_UP) for num in number]
    if isinstance(number, float):
        return Decimal(str(number)).quantize(Decimal('0.' + '0' * ndigits), rounding=ROUND_HALF_UP)


In [16]:
import os
from pathlib import Path
path = Path(os.getcwd()).resolve()
os.path.join(path, 'output')

'C:\\Users\\Light_ouo\\Documents\\Code\\Wind_Exporter_lite\\output'

In [14]:
def create_folder(choice_data):
    path = './output/{}'.format(choice_data)
    if not os.path.exists(path):
        os.makedirs(path)
    if not os.path.exists(path+'/data'):
        os.makedirs(path+'/data')
    if not os.path.exists(path+'/picture'):
        os.makedirs(path+'/picture')
create_folder('2021-12-05')

In [15]:
os.path.dirname(os.path.abspath(__file__))

NameError: name '__file__' is not defined